In [1]:
import nltk
#nltk.download("all")

In [108]:
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import os
import gensim
from gensim import corpora
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from collections import Counter
from gensim.models import Word2Vec
from tqdm.notebook import tqdm


# Important: #
### You need to be in the code folder of the working directory (ML-for-NLP-main\Code) for the following code to work ###

### Challenge: Use the senator speeches in the folder 105-extracted-date and use cosine similarity to find whose senator speech is closest to senator Biden. Use sen105kh_fix.csv and/or Wikipedia to validate your findings (i.e., understand if the most similar speeches are senators from the same state and/party).  Describe your findings. (Compare with the outcome you got using cosine similarity.)

In [3]:
Inputs_path = '../Inputs'
items = os.listdir(Inputs_path)
items

['105-extracted-date',
 '106-extracted-date',
 'AmazonShort.csv',
 'anya_file.pdf',
 'app20170223.pdf',
 'app20170295.pdf',
 'articles_bbc_2018_01_30.csv',
 'Audio',
 'bbc_news.txt',
 'droplist.txt',
 'FoxNews_all_tweets.csv',
 'glove',
 'hello-world.png',
 'imdb_reviews.txt',
 'indexed',
 'Mall_Customers.csv',
 'Motivational1.txt',
 'Motivational2.txt',
 'Motivational3.txt',
 'Motivational4.txt',
 'nltk_stopwords.txt',
 'nytimes_all_tweets.csv',
 'sen105kh_fix.csv',
 'smsspamcollection',
 'Social_Network_Ads.csv',
 'spam-reduced',
 'Syllabus-ML-for-NLP.docx',
 'text_to_emotion.csv',
 'Wine.csv',
 'ПАД_Комп науки_0001.pdf']

In [4]:
import os
path_105 = '../Inputs/105-extracted-date'
path_106 = '../Inputs/106-extracted-date'
path_droplist = '../Inputs/droplist.txt'
items = os.listdir(Inputs_path) #This gives me a list of both files and folders in dir

for name in items:
    if os.path.isfile(Inputs_path + '/' + name) == True:
        items.remove(name)

In [5]:
all_files = os.listdir(path_105)

In [6]:
all_files

['105-abraham-mi.txt',
 '105-akaka-hi.txt',
 '105-allard-co.txt',
 '105-ashcroft-mo.txt',
 '105-baucus-mt.txt',
 '105-bennett-ut.txt',
 '105-biden-de.txt',
 '105-bingaman-nm.txt',
 '105-bond-mo.txt',
 '105-boxer-ca.txt',
 '105-breaux-la.txt',
 '105-brownback-ks.txt',
 '105-bryan-nv.txt',
 '105-bumpers-ar.txt',
 '105-burns-mt.txt',
 '105-byrd-wv.txt',
 '105-campbell-co.txt',
 '105-chafee-ri.txt',
 '105-cleland-ga.txt',
 '105-coats-in.txt',
 '105-cochran-ms.txt',
 '105-collins-me.txt',
 '105-conrad-nd.txt',
 '105-coverdell-ga.txt',
 '105-craig-id.txt',
 '105-damato-ny.txt',
 '105-daschle-sd.txt',
 '105-dewine-oh.txt',
 '105-dodd-ct.txt',
 '105-domenici-nm.txt',
 '105-dorgan-nd.txt',
 '105-durbin-il.txt',
 '105-enzi-wy.txt',
 '105-faircloth-nc.txt',
 '105-feingold-wi.txt',
 '105-feinstein-ca.txt',
 '105-ford-ky.txt',
 '105-frist-tn.txt',
 '105-glenn-oh.txt',
 '105-gorton-wa.txt',
 '105-graham-fl.txt',
 '105-gramm-tx.txt',
 '105-grams-mn.txt',
 '105-grassley-ia.txt',
 '105-gregg-nh.txt',
 

In [7]:
doc_list = []
for file in all_files:
   # open the file and then call .read() to get the text
   if file.startswith('105'):
      with open(os.path.join(path_105, file),encoding = "ISO-8859-1") as f:
         text = f.read()
         doc_list.append(text)

In [8]:
# Test whether we got everthing
print('Number of raw files: %i' %len(all_files))
print('Number of files imported: %i' %len(doc_list))

Number of raw files: 100
Number of files imported: 100


In [9]:
#Define stop words 
stop_words = stopwords.words('english')
stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'would', 'can']
stopwords_other = ['one', 'mr', 'mrs', 'ms', 'also', 'copyright', 'something','doc','docno','text']
stop_words = stopwords.words('English') + stopwords_verbs + stopwords_other

In [10]:
from nltk.corpus import wordnet
# original example can be found here https://stackoverflow.com/a/15590384

def wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [11]:
#from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
def get_lemma(element):
    for word, pos in pos_tag(element.split()):
        result_pos=wordnet_pos(pos)
        if result_pos != None:
            return WordNetLemmatizer().lemmatize(word, result_pos)
        else:
            return word

In [12]:
def text_preprocesser(text):
    text= re.sub(r'\W',' ', text)
    text= re.sub(r'\d', '', text)
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [word for word in tokens if len(word)>=3]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text 

In [13]:
def text_preprocesser_with_lema(text):
    text= re.sub(r'\W',' ', text)
    text= re.sub(r'\d', '', text)
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [word for word in tokens if len(word)>=3]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [get_lemma(t) for t in filtered_tokens]
    preprocessed_text = ' '.join(stems)
    return preprocessed_text 

In [14]:
def text_preprocesser_with_lema_tokenized(text):
    text= re.sub(r'\W',' ', text)
    text= re.sub(r'\d', '', text)
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [word for word in tokens if len(word)>=3]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [get_lemma(t) for t in filtered_tokens]
    
    return stems

### First, we use tf-idf and then doc2vec

In [26]:
tfidf_vectorizer = TfidfVectorizer(preprocessor = text_preprocesser_with_lema, min_df =2 )

In [27]:
tfidf_vectorizer

TfidfVectorizer(min_df=2,
                preprocessor=<function text_preprocesser_with_lema at 0x000001F880C0C3A0>)

In [28]:
tfidf = tfidf_vectorizer.fit_transform(doc_list)

In [15]:
senators = [x.replace('.txt', '').replace('105-', '') for x in all_files]
#senators = senators[5:10]
senators

['abraham-mi',
 'akaka-hi',
 'allard-co',
 'ashcroft-mo',
 'baucus-mt',
 'bennett-ut',
 'biden-de',
 'bingaman-nm',
 'bond-mo',
 'boxer-ca',
 'breaux-la',
 'brownback-ks',
 'bryan-nv',
 'bumpers-ar',
 'burns-mt',
 'byrd-wv',
 'campbell-co',
 'chafee-ri',
 'cleland-ga',
 'coats-in',
 'cochran-ms',
 'collins-me',
 'conrad-nd',
 'coverdell-ga',
 'craig-id',
 'damato-ny',
 'daschle-sd',
 'dewine-oh',
 'dodd-ct',
 'domenici-nm',
 'dorgan-nd',
 'durbin-il',
 'enzi-wy',
 'faircloth-nc',
 'feingold-wi',
 'feinstein-ca',
 'ford-ky',
 'frist-tn',
 'glenn-oh',
 'gorton-wa',
 'graham-fl',
 'gramm-tx',
 'grams-mn',
 'grassley-ia',
 'gregg-nh',
 'hagel-ne',
 'harkin-ia',
 'hatch-ut',
 'helms-nc',
 'hollings-sc',
 'hutchinson-ar',
 'hutchison-tx',
 'inhofe-ok',
 'inouye-hi',
 'jeffords-vt',
 'johnson-sd',
 'kempthorne-id',
 'kennedy-ma',
 'kerrey-ne',
 'kerry-ma',
 'kohl-wi',
 'kyl-az',
 'landrieu-la',
 'lautenberg-nj',
 'leahy-vt',
 'levin-mi',
 'lieberman-ct',
 'lott-ms',
 'lugar-in',
 'mack-fl',
 

In [30]:
df= pd.DataFrame(tfidf.toarray().transpose(), index=tfidf_vectorizer.get_feature_names(), columns = [senators])

C:\Users\Ramzi\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#### Cosine similarity

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cos_sim_list = []
for (colname,colval) in df.loc[:, df.columns != 'biden-de'].iteritems():
    sim_value = cosine_similarity(colval.values.reshape(1, -1), df["biden-de"].values.reshape(1, -1))
    cos_sim_list.append(*sim_value)
    results = pd.DataFrame(zip(cos_sim_list,df.columns), columns = ["cosine similarity","senator"])
    results['cosine similarity'] = results['cosine similarity'].str[0]

In [33]:
results.sort_values(by="cosine similarity", ascending = False)

,cosine similarity,senator
6,1.000000,"(biden-de,)"
66,0.744981,"(lieberman-ct,)"
81,0.743000,"(roberts-ks,)"
61,0.724641,"(kyl-az,)"
59,0.715084,"(kerry-ma,)"
...,...,...
16,0.524192,"(campbell-co,)"
1,0.509615,"(akaka-hi,)"
36,0.504746,"(ford-ky,)"
92,0.463002,"(stevens-ak,)"


The senator whose speeches resemble Biden the most is Lieberman from Connecticut. This can be explained by several factors:
- both of them are Democrats
- both represented small East Coast Blue states (Connecticut & Deleware)
- both were born in 1942
- both studied law
- both worked on foregin policy related topics in their respective committees

### doc2vec 

In [16]:
text_preprocesser_with_lema_tokenized(doc_list[0])

['abraham',
 'abraham',
 'president',
 'debate',
 'final',
 'passage',
 'omnibus',
 'appropriation',
 'bill',
 'american',
 'competitiveness',
 'workforce',
 'improvement',
 'act',
 'include',
 'title',
 'subdivision',
 'ask',
 'unanimous',
 'consent',
 'number',
 'document',
 'print',
 'record',
 'include',
 'two',
 'document',
 'receive',
 'administration',
 'negotiation',
 'whose',
 'inclusion',
 'seek',
 'help',
 'illuminate',
 'meaning',
 'provision',
 'legislation',
 'key',
 'point',
 'document',
 'change',
 'july',
 'version',
 'september',
 'version',
 'copy',
 'submit',
 'change',
 'marked',
 'redline',
 'marking',
 'unfortunately',
 'however',
 'submit',
 'copy',
 'version',
 'copy',
 'fax',
 'marking',
 'appear',
 'effect',
 'make',
 'september',
 'version',
 'unintelligible',
 'result',
 'printing',
 'garble',
 'contain',
 'marking',
 'show',
 'change',
 'accordingly',
 'ask',
 'correct',
 'version',
 'document',
 'submit',
 'appear',
 'final',
 'issue',
 'record',
 'congre

In [17]:
speeches_df = pd.DataFrame()

In [18]:
speeches_df['senator'] = senators   
speeches_df['speeches_raw'] = doc_list

In [23]:
speeches = []
for text in doc_list:
    speeches.append(text_preprocesser_with_lema_tokenized(text))

In [24]:
speeches[0]

['abraham',
 'abraham',
 'president',
 'debate',
 'final',
 'passage',
 'omnibus',
 'appropriation',
 'bill',
 'american',
 'competitiveness',
 'workforce',
 'improvement',
 'act',
 'include',
 'title',
 'subdivision',
 'ask',
 'unanimous',
 'consent',
 'number',
 'document',
 'print',
 'record',
 'include',
 'two',
 'document',
 'receive',
 'administration',
 'negotiation',
 'whose',
 'inclusion',
 'seek',
 'help',
 'illuminate',
 'meaning',
 'provision',
 'legislation',
 'key',
 'point',
 'document',
 'change',
 'july',
 'version',
 'september',
 'version',
 'copy',
 'submit',
 'change',
 'marked',
 'redline',
 'marking',
 'unfortunately',
 'however',
 'submit',
 'copy',
 'version',
 'copy',
 'fax',
 'marking',
 'appear',
 'effect',
 'make',
 'september',
 'version',
 'unintelligible',
 'result',
 'printing',
 'garble',
 'contain',
 'marking',
 'show',
 'change',
 'accordingly',
 'ask',
 'correct',
 'version',
 'document',
 'submit',
 'appear',
 'final',
 'issue',
 'record',
 'congre

In [25]:
speeches_df['speeches'] = speeches

In [26]:
speeches_df

,senator,speeches_raw,speeches
0,abraham-mi,<DOC>\n<DOCNO>105-abraham-mi-1-19981112</DOCNO...,"[abraham, abraham, president, debate, final, p..."
1,akaka-hi,<DOC>\n<DOCNO>105-akaka-hi-1-19981021</DOCNO>\...,"[akaka, akaka, president, pleased, senate, pas..."
2,allard-co,<DOC>\n<DOCNO>105-allard-co-1-19981009</DOCNO>...,"[allard, allard, president, rise, remark, conc..."
3,ashcroft-mo,<DOC>\n<DOCNO>105-ashcroft-mo-1-19981021</DOCN...,"[ashcroft, ashcroft, president, senate, order,..."
4,baucus-mt,<DOC>\n<DOCNO>105-baucus-mt-1-19981021</DOCNO>...,"[baucus, baucus, understand, house, sent, sena..."
...,...,...,...
95,thurmond-sc,<DOC>\n<DOCNO>105-thurmond-sc-1-19981021</DOCN...,"[thurmond, thurmond, president, senate, consid..."
96,torricelli-nj,<DOC>\n<DOCNO>105-torricelli-nj-1-19981012</DO...,"[torricelli, torricelli, president, thank, sen..."
97,warner-va,<DOC>\n<DOCNO>105-warner-va-1-19981021</DOCNO>...,"[warner, warner, past, two, week, senate, arm,..."
98,wellstone-mn,<DOC>\n<DOCNO>105-wellstone-mn-1-19981021</DOC...,"[wellstone, wellstone, president, today, call,..."


In [29]:
biden = speeches_df[speeches_df['senator']=='biden-de']

In [31]:
# index of biden's speech
a=speeches_df[speeches_df['senator']=='biden-de'].index.values
biden_index=a[0]

In [45]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)


In [46]:
text=[]
for i in speeches:
    text.append(TaggedDocument(words=i, tags=[speeches.index(i)]))

In [47]:
model.build_vocab(text)

In [48]:
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

In [49]:
ranks=[]
second_ranks=[]
inferred_vector = model.infer_vector(text[biden_index].words)
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
rank = [docid for docid, sim in sims].index(biden_index)
ranks.append(rank)

second_ranks.append(sims[1])

In [50]:
sims

[(6, 0.9976264238357544),
 (95, 0.5585360527038574),
 (42, 0.55560302734375),
 (27, 0.5219978094100952),
 (97, 0.519589900970459),
 (15, 0.49996668100357056),
 (74, 0.4987369477748871),
 (38, 0.47682005167007446),
 (81, 0.47177854180336),
 (89, 0.46267351508140564),
 (68, 0.45341765880584717),
 (22, 0.4502507150173187),
 (77, 0.4345338046550751),
 (66, 0.43106499314308167),
 (61, 0.42781639099121094),
 (34, 0.4265343248844147),
 (24, 0.41296032071113586),
 (48, 0.4080500900745392),
 (19, 0.4077724814414978),
 (11, 0.4015531539916992),
 (58, 0.3960985839366913),
 (47, 0.3911485970020294),
 (55, 0.3844311237335205),
 (64, 0.3810671865940094),
 (65, 0.3624044358730316),
 (20, 0.3512609302997589),
 (94, 0.34858494997024536),
 (3, 0.3424389958381653),
 (29, 0.34226086735725403),
 (7, 0.3403148353099823),
 (4, 0.3289004862308502),
 (71, 0.32513996958732605),
 (91, 0.32234469056129456),
 (0, 0.31734102964401245),
 (35, 0.3173215985298157),
 (26, 0.3151986300945282),
 (10, 0.31474706530570984)

In [51]:
speeches_df['senator'][sims[1][0]]

'thurmond-sc'

In [52]:
speeches_df['senator'][sims[0][0]]

'biden-de'

Naturally, the most similar speech to biden's is the same speech, that's why we take the second one which is thurmond

This is the top 10 ranking:

In [54]:
for i in range(10):
    print(speeches_df['senator'][sims[i+1][0]])

thurmond-sc
grams-mn
dewine-oh
warner-va
byrd-wv
moynihan-ny
glenn-oh
roberts-ks
smith-or
lugar-in


It is quite different from the ranking given by tf-idf

### Challenge: Are the top ten words mentioned by Biden in the 105 congress (after stopwords removal). Find the 10 most simlar words generated using word2vec. Is the most frequent bigram included in the the list of most similar word?

In [84]:
range(len(speeches)-1)

range(0, 99)

In [90]:
# we need the preprocessed speech that is currently in a list to be in one string
speech_list = []
for i in range(len(speeches)):
    speech = ' '.join(speeches[i])
    speech_list.append(speech) 


In [91]:
speeches_df['processed_speeches'] = speech_list

In [92]:
speeches_df

,senator,speeches_raw,speeches,processed_speeches
0,abraham-mi,<DOC>\n<DOCNO>105-abraham-mi-1-19981112</DOCNO...,"[abraham, abraham, president, debate, final, p...",abraham abraham president debate final passage...
1,akaka-hi,<DOC>\n<DOCNO>105-akaka-hi-1-19981021</DOCNO>\...,"[akaka, akaka, president, pleased, senate, pas...",akaka akaka president pleased senate pass vete...
2,allard-co,<DOC>\n<DOCNO>105-allard-co-1-19981009</DOCNO>...,"[allard, allard, president, rise, remark, conc...",allard allard president rise remark concern au...
3,ashcroft-mo,<DOC>\n<DOCNO>105-ashcroft-mo-1-19981021</DOCN...,"[ashcroft, ashcroft, president, senate, order,...",ashcroft ashcroft president senate order hear ...
4,baucus-mt,<DOC>\n<DOCNO>105-baucus-mt-1-19981021</DOCNO>...,"[baucus, baucus, understand, house, sent, sena...",baucus baucus understand house sent senate sub...
...,...,...,...,...
95,thurmond-sc,<DOC>\n<DOCNO>105-thurmond-sc-1-19981021</DOCN...,"[thurmond, thurmond, president, senate, consid...",thurmond thurmond president senate considers b...
96,torricelli-nj,<DOC>\n<DOCNO>105-torricelli-nj-1-19981012</DO...,"[torricelli, torricelli, president, thank, sen...",torricelli torricelli president thank senator ...
97,warner-va,<DOC>\n<DOCNO>105-warner-va-1-19981021</DOCNO>...,"[warner, warner, past, two, week, senate, arm,...",warner warner past two week senate arm service...
98,wellstone-mn,<DOC>\n<DOCNO>105-wellstone-mn-1-19981021</DOC...,"[wellstone, wellstone, president, today, call,...",wellstone wellstone president today call atten...


In [103]:
biden=speeches_df.loc[speeches_df["senator"] == "biden-de", "speeches"].item()

In [104]:
biden_dict=Counter(biden)

In [105]:
most_used= sorted(biden_dict.items(), key=lambda kv: kv[1], reverse=True)[1:11]

In [106]:
most_used

[('president', 1548),
 ('state', 1160),
 ('senator', 1104),
 ('nato', 985),
 ('year', 980),
 ('say', 705),
 ('time', 700),
 ('united', 653),
 ('go', 610),
 ('bill', 580)]

In [109]:
most_similar={}
model = Word2Vec(sentences=most_used, vector_size=100, window=5, min_count=1, workers=4)

In [110]:
for i in most_used:
    vector = model.wv[i]
    most_similar[i[0]]=model.wv.most_similar(i[0], topn=10)

In [111]:
most_similar

{'president': [('nato', 0.31900981068611145),
  (700, 0.16207121312618256),
  (1104, 0.1107415035367012),
  ('say', 0.09731774777173996),
  (985, 0.09673888981342316),
  (980, 0.08635810017585754),
  (580, 0.002436041831970215),
  ('united', 0.0007033858564682305),
  (610, 0.0004826862714253366),
  ('state', -0.0017842884408310056)],
 'state': [(700, 0.17826788127422333),
  ('senator', 0.13149337470531464),
  (985, 0.07497557252645493),
  ('bill', 0.0679759606719017),
  (1104, 0.04157736524939537),
  ('say', 0.04130806028842926),
  ('go', 0.041194360703229904),
  (980, 0.012979956343770027),
  ('time', 0.006598414853215218),
  ('president', -0.0017842951929196715)],
 'senator': [('say', 0.1669394075870514),
  (1160, 0.1388825923204422),
  ('state', 0.13149337470531464),
  ('go', 0.07171673327684402),
  (1548, 0.0641021654009819),
  (985, 0.06059250235557556),
  (653, 0.04768550023436546),
  ('time', 0.04410773515701294),
  (980, 0.02000463753938675),
  (1104, 0.01923765428364277)],
 'n

In [112]:
bigrams = zip(biden, biden[1:])
counts = Counter(bigrams)
most_common=counts.most_common
most_common(10)

[(('biden', 'biden'), 830),
 (('united', 'state'), 479),
 (('biden', 'president'), 387),
 (('foreign', 'policy'), 161),
 (('united', 'nation'), 159),
 (('chemical', 'weapon'), 155),
 (('nato', 'enlargement'), 116),
 (('last', 'year'), 115),
 (('state', 'department'), 111),
 (('foreign', 'relation'), 110)]